In [1]:
import torch
import pandas as pd
import numpy as np
import gzip
import itertools
import multiprocessing
import csv
import pickle
from sklearn.metrics.pairwise import cosine_similarity as cosine
from sklearn.metrics import mean_squared_error as mse
from tqdm import tqdm, tqdm_notebook
from multiprocessing import Manager, Pool
from scipy.spatial.distance import cdist
from numpy.linalg import norm
from scipy.stats import spearmanr, pearsonr
import random

In [2]:
# UNIPROT data for mapping between UNIPROT accession numbers and UNIPROT entry names
uniprot_metadata_directory = "/media/DATA/serbulent/DATA/Thesis/ReviewPaper/Uniprot/"
uniprot_metadata_file_path = uniprot_metadata_directory + "uniprot_human_all.tab"
uniprot_vars = ['Entry','Entry name','Status','Protein names','Gene names','Organism','Length','Annotation' ]
uniprot_df = pd.read_csv(uniprot_metadata_file_path, sep='\t')

In [3]:
representationFile = '/media/DATA/serbulent/DATA/Thesis/ReviewPaper/results/SeqVec/seqvec_calculated_human_protein_vectors.npy'
SeqVec = np.load(representationFile)

In [4]:
SeqVec_embedding = SeqVec.item()

In [5]:
SeqVec_embedding['1433B_HUMAN']#.tolist()


tensor([-0.0871, -0.0579, -0.1127,  ..., -0.0477,  0.0703,  0.0759])

In [6]:
len(SeqVec_embedding['1433B_HUMAN'])

1024

In [7]:
SeqVecDF = pd.DataFrame(columns=['Entry', 'Vector'])
i=0
for entry_name,vector in tqdm_notebook(SeqVec_embedding.items()):
    try:
        entry = uniprot_df[uniprot_df['Entry name'] == entry_name]['Entry'].item()
        SeqVecDF.loc[i] = [entry,vector.tolist()]
        i+=1
    except:
        print(entry_name)
        print(uniprot_df[uniprot_df['Entry name'] == entry_name])
        pass

In [8]:
SeqVecDF[0:10]

,Entry,Vector
0,P31946,"[-0.08708333224058151, -0.05792386829853058, -..."
1,P62258,"[-0.08087854087352753, -0.06604673713445663, -..."
2,Q04917,"[-0.09406778216362, -0.025370510295033455, -0...."
3,P61981,"[-0.08130422234535217, -0.04338306188583374, -..."
4,P31947,"[-0.07861586660146713, -0.011337828822433949, ..."
5,P27348,"[-0.13528062403202057, -0.039607856422662735, ..."
6,P63104,"[-0.09760383516550064, -0.045363813638687134, ..."
7,P30443,"[-0.01358085498213768, -0.14518339931964874, -..."
8,P01892,"[-0.011920960620045662, -0.11270976811647415, ..."
9,P04439,"[-0.02001873403787613, -0.131582111120224, -0...."


In [21]:
SeqVecDF.to_csv("/media/DATA/serbulent/DATA/Thesis/ReviewPaper/results/embedding_dataframes/SeqVec_dataframe.csv")

In [10]:
SeqVecDF.to_pickle("/media/DATA/serbulent/DATA/Thesis/ReviewPaper/results/embedding_dataframes/SeqVec_dataframe.pkl")

In [6]:
SeqVecProteinList = list(SeqVec_embedding.keys())

In [7]:
'''#This part was used to be sure parallel and sequential versions gives same results
cosine_distance_list1 = []
real_distance_list1 = []

similarityMatrixFileName = ""
#similarityMatrixFileName = "/media/DATA/serbulent/Code/Thesis/ReviewPaper/preprocess/human_MF_protienSimilarityMatrix.csv"
#similarityMatrixFileName = "/media/DATA/serbulent/Code/Thesis/ReviewPaper/preprocess/human_BP_protienSimilarityMatrix.csv"
similarityMatrixFileName = "/media/DATA/serbulent/Code/Thesis/ReviewPaper/preprocess/human_CC_protienSimilarityMatrix.csv"

human_proteinSimilarityMatrix = pd.read_csv(similarityMatrixFileName)
human_proteinSimilarityMatrix.set_index(human_proteinSimilarityMatrix.columns, inplace = True)

proteinListTmp = human_proteinSimilarityMatrix.columns[0:10]
for i,protein1 in tqdm_notebook(enumerate(proteinListTmp)):
    for j in range(len(proteinListTmp)):
        if j>i:
            protein2 = proteinListTmp[j]
            #print((protein1,protein2))
            prot1name = uniprot_df.query("Entry == @protein1")['Entry name'].item()
            prot2name = uniprot_df.query("Entry == @protein2")['Entry name'].item()
            #if protein1 in LearnedVec_GenesList and protein2 in LearnedVec_GenesList:
            prot1vec = np.asarray(SeqVec_embedding[prot1name].tolist())
            prot2vec = np.asarray(SeqVec_embedding[prot2name].tolist())
            #cosine will return in shape of input vectors first dimension
            cosine_dist = cosine(prot1vec.reshape(1,-1),prot2vec.reshape(1,-1)).item()
            cosine_distance_list1.append(cosine_dist)
            real_distance_list1.append(human_proteinSimilarityMatrix.loc[protein1,protein2])

print(len(cosine_distance_list1))
print(spearmanr(real_distance_list1,cosine_distance_list1))

# MF corr 0.5081127442940586
# BP corr 0.2608781581836211
# CC corr 0.40952663506626436
'''

'#This part was used to be sure parallel and sequential versions gives same results\ncosine_distance_list1 = []\nreal_distance_list1 = []\n\nsimilarityMatrixFileName = ""\n#similarityMatrixFileName = "/media/DATA/serbulent/Code/Thesis/ReviewPaper/preprocess/human_MF_protienSimilarityMatrix.csv"\n#similarityMatrixFileName = "/media/DATA/serbulent/Code/Thesis/ReviewPaper/preprocess/human_BP_protienSimilarityMatrix.csv"\nsimilarityMatrixFileName = "/media/DATA/serbulent/Code/Thesis/ReviewPaper/preprocess/human_CC_protienSimilarityMatrix.csv"\n\nhuman_proteinSimilarityMatrix = pd.read_csv(similarityMatrixFileName)\nhuman_proteinSimilarityMatrix.set_index(human_proteinSimilarityMatrix.columns, inplace = True)\n\nproteinListTmp = human_proteinSimilarityMatrix.columns[0:10]\nfor i,protein1 in tqdm_notebook(enumerate(proteinListTmp)):\n    for j in range(len(proteinListTmp)):\n        if j>i:\n            protein2 = proteinListTmp[j]\n            #print((protein1,protein2))\n            prot1n

In [8]:
# define similarity_list and proteinList as global variables
proteinList = []
manager = Manager()
similarity_list = manager.list()
proteinListNew = manager.list()

def parallelSimilarity(paramList):
    i = paramList[0]
    j = paramList[1] 
    aspect = paramList[2]
    if j>i:
        protein1 = proteinListNew[i]
        protein2 = proteinListNew[j]     
        prot1name = uniprot_df.query("Entry == @protein1")['Entry name'].item()
        prot2name = uniprot_df.query("Entry == @protein2")['Entry name'].item()     
        prot1vec = np.asarray(SeqVec_embedding[prot1name].tolist())
        prot2vec = np.asarray(SeqVec_embedding[prot2name].tolist())
        #cosine will return in shape of input vectors first dimension
        cos = cosine(prot1vec.reshape(1,-1),prot2vec.reshape(1,-1)).item()
        manhattanDist = cdist(prot1vec.reshape(1,-1), prot2vec.reshape(1,-1), 'cityblock')
        manhattanDistNorm = manhattanDist/(norm(prot1vec,1) + norm(prot2vec,1))
        euclideanDist = cdist(prot1vec.reshape(1,-1), prot2vec.reshape(1,-1), 'euclidean')
        euclideanDistNorm = euclideanDist/(norm(prot1vec,2) + norm(prot2vec,2)) 
        #print([(prot1name,prot2name),(prot1vec,prot2vec)])
        #print((prot1name,prot2name))
        #print()
        #print((cos,euclideanDist,norm(prot1vec,2),norm(prot2vec,2)))
        real = paramList[3]
        #real = human_protienSimilarityMatrix.loc[protein1,protein2]
        # To ensure real and calculated values appended to same postion they saved similtanously and then decoupled
        #Similarity = 1-distance
        similarity_list.append((real,1-cos,1-manhattanDistNorm.item(),1-euclideanDistNorm.item()))
    return similarity_list


In [14]:
real_distance_list = []
cosine_distance_list = []
euclidian_distance_list = []

def calculateMSEforOntology(aspect,sparse=False):
    
    #Clear lists before each aspect
    similarity_list[:] = []
    proteinListNew[:] = []

    #similarityMatrixFileName = "/media/DATA/serbulent/Code/Thesis/ReviewPaper/preprocess/human_"+aspect+"_protienSimilarityMatrix.csv"
    similarityMatrixFileName = "/media/DATA/serbulent/Code/Thesis/ReviewPaper/preprocess/human_"\
    +aspect+"_proteinSimilarityMatrix_for_highest_annotated_500_proteins.csv"
  
    human_proteinSimilarityMatrix = pd.read_csv(similarityMatrixFileName)
    human_proteinSimilarityMatrix.set_index(human_proteinSimilarityMatrix.columns, inplace = True)
    proteinList = human_proteinSimilarityMatrix.columns
    
    #proteinListNew is referanced using Manager
    for prot in proteinList:
        proteinListNew.append(prot)
    if sparse:
        #sparsified_similarities = np.load("SparsifiedSimilarites_for_highest_500.npy")
        sparsified_similarity_coordinates = np.load("SparsifiedSimilarityCoordinates_"+aspect+"_for_highest_500.npy")
        protParamList = sparsified_similarity_coordinates
    else:     
        i = range(len(proteinList))
        j = range(len(proteinList))
        protParamList = list(itertools.product(i,j))
    protParamListNew = []
    # Prepare parameters for parallel processing these parameters will be 
    # used concurrently by different processes
    for tup in tqdm_notebook(protParamList):
        i = tup[0]
        j = tup[1]
        
        if sparse:
            protein1 = proteinListNew[i]
            protein2 = proteinListNew[j]
            real = human_proteinSimilarityMatrix.loc[protein1,protein2]
            tupNew = (tup[0],tup[1],aspect,real)
            protParamListNew.append(tupNew)
        else:
            if j > i:
                protein1 = proteinListNew[i]
                protein2 = proteinListNew[j]
                real = human_proteinSimilarityMatrix.loc[protein1,protein2]
                tupNew = (tup[0],tup[1],aspect,real)
                protParamListNew.append(tupNew)

    total_task_num=len(protParamListNew)
    pool = Pool()
    similarity_listRet = []
    for similarity_listRet in tqdm_notebook(pool.imap_unordered(parallelSimilarity, protParamListNew), total=total_task_num):
        pass
    real_distance_list = [value[0] for value in similarity_listRet]
    cosine_distance_list = [value[1] for value in similarity_listRet]
    manhattan_distance_list = [value[2] for value in similarity_listRet]
    euclidian_distance_list = [value[3] for value in similarity_listRet]

    #mseValue = mse(real_distance_list,cosine_distance_list)
    cosineCorr = spearmanr(real_distance_list, cosine_distance_list)
    manhattanCorr = spearmanr(real_distance_list, manhattan_distance_list)
    euclidianCorr = spearmanr(real_distance_list, euclidian_distance_list)
    
    random.seed(42)
    random_list = []
    for i in range(len(real_distance_list)):
        random_list.append(random.uniform(0, 1))
    
    if sparse:
        cosine_randomCorr = spearmanr(cosine_distance_list, random_list)
        manhattan_randomCorr = spearmanr(manhattan_distance_list, random_list)
        euclidian_randomCorr = spearmanr(euclidian_distance_list, random_list)
        print("Cosine Random Correlation for "+aspect+" is " + str(cosine_randomCorr))
        print("Manhattan Random Correlation for "+aspect+" is " + str(manhattan_randomCorr))
        print("Euclidian Random Correlation for "+aspect+" is " + str(euclidian_randomCorr))
    
    
    print("Cosine Correlation for "+aspect+" is " + str(cosineCorr))
    print("Manhattan Correlation for "+aspect+" is " + str(manhattanCorr))
    print("Euclidian Correlation for "+aspect+" is " + str(euclidianCorr))

    if sparse:
        return (cosineCorr,manhattanCorr,euclidianCorr,cosine_randomCorr,manhattan_randomCorr,euclidian_randomCorr)
    else:
        return (cosineCorr,manhattanCorr,euclidianCorr)
    

In [10]:
buffer = "aspect,cosineCorr,cosineCorrPVal,manhattanCorr,manhattanCorrPVal,euclidianCorr,euclidianCorrPVal \n"
saveFileName = "SimilaritySeqVec_highest_200.csv"
f = open(saveFileName,'w')
f.write(buffer)

for aspect in ["MF","BP","CC"]:
    corr = calculateMSEforOntology(aspect) 
    buffer = "" + aspect + ","+ str(corr[0][0])+ ","+ str(corr[0][1])\
    + ","+ str(corr[1][0])+ ","+ str(corr[1][1])+ ","+ str(corr[2][0])+ ","+ str(corr[2][1])+"\n" 
    f = open(saveFileName,'a')
    f.write(buffer) #Give your csv text here.
    ## Python will convert \n to os.linesep
    f.close()
    
# MF corr 0.5081127442940586
# BP corr 0.2608781581836211
# CC corr 0.40952663506626436


Cosine Correlation for MF is SpearmanrResult(correlation=-0.24873222054197036, pvalue=2.5811219068825022e-278)
Manhattan Correlation for MF is SpearmanrResult(correlation=0.26418423281003517, pvalue=0.0)
Euclidian Correlation for MF is SpearmanrResult(correlation=0.2252384890919629, pvalue=2.750064472805622e-227)



Cosine Correlation for BP is SpearmanrResult(correlation=-0.16513958051946254, pvalue=1.1603678520583488e-121)
Manhattan Correlation for BP is SpearmanrResult(correlation=0.137725316255257, pvalue=7.364325262820173e-85)
Euclidian Correlation for BP is SpearmanrResult(correlation=0.1544198303253722, pvalue=1.919415836712509e-106)



Cosine Correlation for CC is SpearmanrResult(correlation=-0.07212022480359531, pvalue=2.2796758864299136e-24)
Manhattan Correlation for CC is SpearmanrResult(correlation=0.10505678562475021, pvalue=5.966779303698988e-50)
Euclidian Correlation for CC is SpearmanrResult(correlation=0.06755337431229234, pvalue=1.4300344012119973e-21)


In [15]:
buffer = "aspect,cosineCorr,cosineCorrPVal,manhattanCorr,manhattanCorrPVal,euclidianCorr,euclidianCorrPVal\
,random_cosineCorr,random_cosineCorrPVal,random_manhattanCorr,random_manhattanCorrPVal,random_euclidianCorr,random_euclidianCorrPVal\n"
#saveFileName = "SimilarityGene2Vec.csv"
saveFileName = "Similarity_Sparse_SeqVec_highest_500.csv"
f = open(saveFileName,'w')
f.write(buffer)

for aspect in ["MF","BP","CC"]:
    corr = calculateMSEforOntology(aspect,True) 
    buffer = "" + aspect + ","+ str(corr[0][0])+ ","+ str(corr[0][1])\
    + ","+ str(corr[1][0])+ ","+ str(corr[1][1])+ ","+ str(corr[2][0])+ ","+ str(corr[2][1])+"\n" 
    f = open(saveFileName,'a')
    f.write(buffer) #Give your csv text here.
    ## Python will convert \n to os.linesep
    f.close()


Cosine Random Correlation for MF is SpearmanrResult(correlation=-0.0056228751992551725, pvalue=0.9300810483705607)
Manhattan Random Correlation for MF is SpearmanrResult(correlation=-0.026254728216950848, pvalue=0.6819800771040005)
Euclidian Random Correlation for MF is SpearmanrResult(correlation=-0.0019825763819895487, pvalue=0.9753196630942772)
Cosine Correlation for MF is SpearmanrResult(correlation=-0.3706007178224306, pvalue=1.9978847934803676e-09)
Manhattan Correlation for MF is SpearmanrResult(correlation=0.42379535007446245, pvalue=3.836091946107342e-12)
Euclidian Correlation for MF is SpearmanrResult(correlation=0.3452467942840215, pvalue=2.708871350039662e-08)



Cosine Random Correlation for BP is SpearmanrResult(correlation=0.027234763662773102, pvalue=0.670154672190781)
Manhattan Random Correlation for BP is SpearmanrResult(correlation=-0.018049799800596934, pvalue=0.777744987843581)
Euclidian Random Correlation for BP is SpearmanrResult(correlation=-0.039226023055581205, pvalue=0.5394833830998174)
Cosine Correlation for BP is SpearmanrResult(correlation=-0.35798424524665773, pvalue=7.010724526676903e-09)
Manhattan Correlation for BP is SpearmanrResult(correlation=0.2436484071644948, pvalue=0.00010961135917583849)
Euclidian Correlation for BP is SpearmanrResult(correlation=0.29407890755151506, pvalue=2.5665385676067112e-06)



Cosine Random Correlation for CC is SpearmanrResult(correlation=-0.08845441415270629, pvalue=0.1675383821888638)
Manhattan Random Correlation for CC is SpearmanrResult(correlation=0.06034402565504411, pvalue=0.34693016124761855)
Euclidian Random Correlation for CC is SpearmanrResult(correlation=0.097824543651927, pvalue=0.12675343747881487)
Cosine Correlation for CC is SpearmanrResult(correlation=-0.35245412039265933, pvalue=1.4146094031024226e-08)
Manhattan Correlation for CC is SpearmanrResult(correlation=0.4242066438730631, pvalue=4.026198391961811e-12)
Euclidian Correlation for CC is SpearmanrResult(correlation=0.33420917347346785, pvalue=8.352387507781996e-08)
